In [1]:
:set -XOverloadedLabels -XOverloadedLists
import Topoll.Samplers.Samplers
import Topoll.DistanceMatrix.DistanceMatrix
import Topoll.Complexes.Witness
import Topoll.Complexes.Rips
import Topoll.SimplicialSet
import Topoll.Drawings
import Named
import qualified Data.Vector as V


In [2]:
pts <- sampleSphereUniformly ! #r 10 ! #n 500

In [3]:
dat <- chooseMaxminLandmarks 12 pts 
wComplex nu r = case witnessSet (computeDistanceMatrix dat) ! #nu nu ! #r r of
  Right cc -> cc
  Left err -> error err

rComplex r = ripsSet (computeTotalDistanceMatrix (landmarkPoints dat)) ! #r r

In [ ]:
mapM_  (print . simplicialHomologyOverQ . wComplex) [0,0.5 .. 10] 

In [5]:
vlShow $ drawPersistentHomologies (simplicialHomologyOverQ . wComplex 2) eqSphere 0.5 20

In [ ]:
mapM_  (print . simplicialHomologyOverQ . rComplex) [0,0.5 .. 10] 

In [ ]:
import Data.List (intercalate)
import Text.Printf
toWolfram :: V.Vector Point -> IO ()
toWolfram points = do
  let f = printf "%.3f"
      pointsInBraces = 
        "{" <> intercalate "," 
            (map (\[x,y,z] -> "{" <> f x <> "," <> f y <> ", " <> f z <> "}")
             (V.toList points)) <> "}"
  putStrLn pointsInBraces
toWolfram pts

In [ ]:
{-# LANGUAGE OverloadedStrings #-}

import Data.List (dropWhileEnd)
import Graphics.Vega.VegaLite
import Topoll.ChainComplex.Field
import qualified Data.Text as T

drawPersistentHomologies :: (Float -> [BettiNumber]) -> ([BettiNumber] -> Bool) -> Float -> Int -> VegaLite
drawPersistentHomologies calcHomologies coloringCond step n =
  let homologiesData = map (\r -> (calcHomologies r, r)) $ take n $ enumFromThen 0 step
      h =
        concat $
          [ zipWith
                  ( \bn hn ->
                      dataRow
                        [("r", Number $ realToFrac r),
                         ("b", Number $ realToFrac bn),
                         ("h", Str  $ "H_" <> T.pack (show hn)),
                         ("c", Str $ if coloringCond bns then "1" else  "0")
                         ]
                  )
                  bns
                  [0 :: Integer ..]
            | (bns, r) <- homologiesData
          ]

      d = dataFromRows [] . foldr (.) id h
      enc =
        encoding
          . position X [PName "r", PmType Quantitative, PAxis [AxTickCount n]]
          . position Y [PName "b", PmType Quantitative, PAxis [AxTitleAngle 0, AxTitlePadding 10, AxTickCount (fromIntegral $ maximum $ concatMap fst homologiesData)]]
          . column [FName "h", FmType Nominal, FNoTitle]
          . color [MName "c", MLegend [], MScale [SRange (RStrings ["blue","red"])]]
   in toVegaLite
        [ d [],
          mark Point [],
          enc []
        ]

eqSphere :: [BettiNumber] -> Bool
eqSphere xs = dropWhileEnd (== 0) xs == [1, 0, 1]
